<table>
  <tr>
    <th>
      <img src="https://raw.githubusercontent.com/riinbre-bioinfo/Colab_Biomarkers/main/Biomarkers/images/RIINBRE-Logo.jpg", height = "125", alt="RI-INBRE Logo">
    </th>
    <th>
      <img src="https://raw.githubusercontent.com/riinbre-bioinfo/Colab_Biomarkers/main/Biomarkers/images/MIC_Logo.png", height = "125", alt="RI-INBRE Logo">
    </th>
  </tr>
</table>

# Analysis of Biomedical Data for Biomarker Discovery
## Submodule 9: Biomarker Discovery Using Machine Learning
### Dr. Christopher L. Hemme
### Director, [RI-INBRE Molecular Informatics Core](https://web.uri.edu/riinbre/mic/)
### The University of Rhode Island College of Pharmacy
Last Updated: June 5, 2023

---

## Introduction

The public view of machine learning (influenced by Hollywood) is that of rogue artificial intelligence running wild and causing chaos.  If you watch the news, it sometimes seems like this is true in real life.  In most cases though, machine learning is simply a way to improve the efficiency of completing some set of tasks.  Several of the methods we've covered in this module - linear regression, logistic regression, principal components analysis, k-means clustering - are used in many machine learning algorithms.  The main difference between how we've used those methods previously is that in machine learning, we are taking advantage of the predictive properties of the methods to map new data to existing data to identify patterns in new data.

It can sometimes be difficult to decipher the different buzzwords associated with the field.  Specifically, people are often confused by the terms <b>artificial intelligence (AI)</b>, <b>machine learning (ML)</b>, and <b>deep learning (DL)</b>.  AI is a broad umbrella term that simply means algorithms that can simulate some level of human intelligence, usually for performing some task such as pattern recognition.  ML is a subset of AI that specializes in algorithms that are capable of learning without being explicitly programmed.  DL is a subset of ML that specifically tries to mimic the processes of the human brain to identify and classify patterns in large data sets.  We will focus on simple ML for this submodule.

There are several strategies used in ML, but the two most common are <b>supervised</b> and <b>unsupervised learning</b>.  In supervised learning, we use a subset of labeled data to create a training set to train our algorithm, which can then process new data based on the training set.  In unsupervised learning, the algorithm is attempting to identify new patterns or structures in unlabeled data.

Some examples of how ML is currently or will be used in biomedical science include:

- Diagnose a patient's disease severity based on their tissue proteomics profile
- Predict an individual's susceptibility to disease based on their pattern of genetic variations
- Solve the three-dimensional structure of an unknown protein based on its sequence
- Identify approved drugs which can be repurposed to treat diseases based on chemical structure and omics profiles

In this module we will use some basic ML algorithms to analyze the IRI proteomics data we've previously analyzed.

<div class="alert alert-block alert-info">
<b>&#9995; Tip:</b> Blue boxes will indicate helpful tips.</div>

<div class="alert alert-block alert-warning">
<b>&#127891; Note:</b> Used for interesting asides or notes.
</div>

<div class="alert alert-block alert-success">
<b>&#9997; Reference:</b> This box indicates a reference for an attached figure or table.
</div>

<div class="alert alert-block alert-danger">
<b>&#128721; Caution:</b> A red box indicates potential hazards or pitfalls you may encounter.
</div>

## Load the libraries

This notebook creates a basic template for running a machine learning project in R. 

In [ ]:
# Install packages
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
library('BiocManager')

bioc_packages <- c("limma", "ComplexHeatmap", "M3C")
installed_bioc_packages <- bioc_packages %in% rownames(installed.packages())
if (any(installed_bioc_packages == FALSE)) {BiocManager::install(bioc_packages[!installed_bioc_packages])}

BiocManager::install("preprocessCore")
BiocManager::install("preprocessCore", configure.args="--disable-threading", force = TRUE)

packages <- c("caret", "randomForest", "kernlab", "ggraph", "rattle", "gbm")
installed_packages <- packages %in% rownames(installed.packages())
if (any(installed_packages == FALSE)) {install.packages(packages[!installed_packages])}

if(!require(devtools)) install.packages("devtools")
devtools::install_github("kassambara/factoextra")

In [ ]:
# Load the packages
require('preprocessCore')
require('limma')
require('ComplexHeatmap')
require('plyr')
require('tidyverse')
require('factoextra')
require('caret')
require('kernlab')
require('ggpubr')
require('rattle')
require('dplyr')
require('ggraph')
require('igraph')
require('ggplot2')
require('mlbench')
require('pROC')
require("gbm")

## Load the proteome data and merge with the metadata

We will be loading our experimental object from <b>Submodule07 - Exploratory Proteomic Analysis</b> since it already has our proteomic data merged with our metadata and has been normalized.

In [ ]:
exp_obj <- readRDS(file = "data/exp_obj.rds")

<div class="alert alert-block alert-danger">
<b>&#128721; Caution:</b> Reminder that this version of the experimental object should contain the quantile normalized data from submodule 4.  Use <i>str(exp_obj)</i> if you need to verify that this is the case. 
</div>

In [ ]:
# Read in the normalized data from Submodule 4
proteome_norm <- exp_obj$data$proteomics$norm
metadata <- exp_obj$metadata
head(metadata)

In [ ]:
head(proteome_norm)

Next we will transpose the normalized protein data so that it is in the same orientation as the metadata file. That way we can add metadata columns directly to the data frame. We'll make a new data frame so we don't corrupt the original proteome_norm.

In [ ]:
t_proteome = as.data.frame(t(proteome_norm))
head(t_proteome)

## Define Injury States for Machine Learning

Remember that supervised machine learning involves splitting the dataset into training and testing sets. In order to do supervised machine learning, we need to have an outcome to train the algorithm to predict, in this case, predicting injury state from IRI proteomics data. We will define a sample as injured if it has a SCr concentration above a critical threshold, whereas any sample with a SCr below that will be considered healthy. As we did in <b>Submodule 6: Linear and Logistic Regression for Comparison of Quantitative Biomarkers</b>, we will use the clinically established value of SCr >= 1.  

In the code block below, we end up with a cohort that has approximately 3 times as many healthy samples as injured samples. Class imbalances like this are common in machine learning and especially exaggerated in bioinformatics. We will have to account for this when we build our models.

Just as we did before in <b>Submodule06 - Linear and Logistic Regression Biomarkers</b> we will create a new column labeled **'D'** and give a value to each sample depending on whether SCr levels are greater than 1 but instead of giving 1 and 0 values we will use the terms Healthy vs. Injured.

<div class="alert alert-block alert-info">
<b>&#9995; Tip:</b> In a real-world machine learning scenario, defining labels is often a painstaking process that must be done in conjunction with subject matter experts and frequently involves a prolonged back-and-forth process between the team that generated the data and the analysis team.
</div>

<div class="alert alert-block alert-warning">
<b>&#127891; Note:</b> Some of the machine learning code may take several minutes to run.
</div>
<div class="alert alert-block alert-warning">
<b>&#127891; Note:</b> You will have to load the IRI_Biomarker.csv file into your Colab data folder.
</div>

In [ ]:
# Add SCR/BUN data to the metadata frame
biomarkers = read.csv("data/IRI_Biomarkers.csv")
sample_data = cbind.data.frame(metadata,biomarkers)

# Summary statistics
summary(sample_data)

# Define constant normal values for SCR and BUN
SCR = 1

# Define injury state based on SCR. Since BUN has so many missing values, we will only use SCR. if SCR is greater than 1 label Injured
injury = ifelse((sample_data$SCr > SCR),"Injured","Healthy")
table(injury)
sample_data$D = injury

In [ ]:
# Add the Injured outcome label to the proteomics data frame for training

t_proteome$D = factor(sample_data$D)
head(t_proteome)

## Preprocess and train/test split

One of the core concepts of machine learning is the splitting of data into training and test sets. The training dataset is used to build the model and contains the data itself (proteomics counts in this example) and a set of labels (injury state). The model iterates over the training data, learning the relationships between the input data and output labels. Once the model is trained its predictive ability is evaluated using the test dataset. The test dataset should be data that the model has never seen before but that has come from the same distribution as the training data.

We will split the data into training and test datasets comprised of 80% and 20% of the original data, respectively. The **caret** package provides functionality to do this with the **createDataPartition** function. As machine learning becomes increasingly widely adopted, the ML packages in both R and Python have developed cutting edge helper functions like this, and using them for train/test splitting is a best practice that is highly preferable to hard-coding the split with base R commands.

One benefit of using **createDataPartition** is that it automatically handles class imbalances when splitting on a categorical variable. Remember earlier when we said we had many more healthy samples than injury samples? **createDataPartition** recognizes this and makes sure that the training set has roughly the same ratio of healthy:injured as the training set does.  Class imbalances can still affect model performance. If the class you are trying to predict is a rare event, the model can learn that to improve accuracy.  It's best to assign most predictions to the class that makes up the majority of the data it was trained on. One way to handle this is random oversampling for class imbalance which we'll explore later. 

In [ ]:
# Set the seed so that we end up with the same split each time we run this.
set.seed(3456)

# Split the data based on the variable D.
trainIndex <- createDataPartition(t_proteome$D, p = .8, 
                                  list = FALSE, 
                                  times = 1)

# The output is the indices of samples in the training set
head(trainIndex)

In [ ]:
# Define training and testing as data frames
training = t_proteome[trainIndex,]
testing = t_proteome[-trainIndex,]
dim(training)

## Feature selection

It is not feasible to use every protein in our proteomics screening as a feature in the model. Remember that omics datasets are usually sparse, meaning many of the features  will have low information across samples and will be unhelpful for distinguishing injury from healthy samples. Additionally, from a logistical perspective, it is not reasonable to put several thousand proteins into a laboratory test to use for classification in a real-world setting. With that in mind, we need to find the top features that contain the most information and use those top proteins as variables in the model, as opposed to thousands or tens of thousands that can be measured in a typical mass spectrometry experiment. 

It is important to do this feature selection process on the training set and not the full dataset, otherwise it will bias the features based on information in the test set, which your model and features should not see until model evaluation called 'test leakage'. 

In the code block below, we will build a random forest model using all of the features and rank the features based on their impact on the model's accuracy. This will give us a basis from which to include/exclude features in the training set going forward. We will learn about random forest models and the settings below in more detail soon.

In [ ]:
fitControl_randomforest <- trainControl(# 10-fold cross-validation (CV)
                           method = "repeatedcv",
                           number = 10,
                           # Repeated ten times
                           repeats = 10)

# Build random forest from full dataset
set.seed(123)
# mtry hyperparameter is tuned, ntree is not
mtry <- sqrt(ncol(training))
tunegrid <- expand.grid(.mtry=mtry)
rf_full <- train(D~., 
                      data=training, 
                      method='rf', 
                      metric='Accuracy', 
                      tuneGrid=tunegrid, 
                      trControl=fitControl_randomforest)
print(rf_full)

Listing the important variables with `varImp` can help us see what the top 100 important features are. We can narrow the list further to show only the top 30 important features.

In [ ]:
rfImp = varImp(rf_full)
par(pty = "m")
# Plot the top 100 proteins and top 30 proteins
top100 <- plot(rfImp, top = 100)
top30 <- plot(rfImp, top = 30)

top100
top30

We can see from this plot that not all variables are equally valuable in the model. By selecting the top features, we can hopefully capture the core information that these variables encode while avoiding overfitting the model. 

Let's grab the top 30 proteins and use those to train our models.

In [ ]:
nKeep = 30 # Number of features to keep

# Get the ranked importance into a dataframe
ImpMeasure=data.frame(varImp(rf_full)$importance)
ImpMeasure$Vars<-row.names(ImpMeasure)

# Sort the dataframe and extract the protein names (row names)
keepVars = rownames(ImpMeasure[order(-ImpMeasure$Overall),][1:nKeep,])
# Eliminate the ` character from the gene names since it messes with the matching step below
keepVars = gsub("`","",keepVars)

# Subset the training and test datasets to only the features being used
keep = which(colnames(training) %in% keepVars)
training = training[,c(keep,ncol(training))]
testing = testing[,c(keep,ncol(testing))]

## Train and Test by spot checking a handful of models
### Classification algorithms tested: Decision Tree, Random Forest, GBM, and SVM

In this section, we'll test our prediction with our data using different ML algorithms. It is a best practice to assess a number of algorithms appropriate for your problem. In our case, we have identified a 1:4 class imbalance and <b>decision trees</b> are typically better suited to address this scenario. We will test with three tree-based models and one non-tree-based model and compare the performance across the four classification algorithms. All of the models we will test are supervised learning models that use classification or regression methods. For all of these models we will be using the **caret** package.

In each model you'll see similar steps: 
- Set a random seed number so we have similar stats each time this notebook is run
- Set up the training procedure 
- Determine which hyperparameter(s) and which value sets to try in our training to find the best training model
- Test and make predictions with our test data
- View results

<div class="alert alert-block alert-warning">
<b>&#127891; Note:</b> Model accuracy, while intuitive and easy to understand (thus its use in the evaluations below), is not necessarily the best metric to use for imbalanced classification problems. Depending on the problem domain you may need to optimize for other measures. See <a href="https://machinelearningmastery.com/tour-of-evaluation-metrics-for-imbalanced-classification/">https://machinelearningmastery.com/tour-of-evaluation-metrics-for-imbalanced-classification/</a> for more information.
</div>

## Decision Tree Models

As mentioned before, a <b>Decision Tree</b> is a type of supervised machine learning (as are the rest of the models in this submodule) that is used to make a prediction when asked a direct question based on categories within the dataset.  Decision trees go through the predictor variables and continuously split the data like in the example below.  When plotted it shows a single tree or hierarchical diagram that is easy to interpret.  Some terms to remember when looking at the tree are:

- **Root node:** The base of the decision tree.
- **Splitting:** The process of dividing a node into multiple sub-nodes.
- **Decision node:** When a sub-node is further split into additional sub-nodes.
- **Leaf node:** When a sub-node does not further split into additional sub-nodes; represents possible outcomes.
- **Pruning:** The process of removing sub-nodes of a decision tree.
- **Branch:** A subsection of the decision tree consisting of multiple nodes.

<div>
  <img src="https://raw.githubusercontent.com/riinbre-bioinfo/Colab_Biomarkers/main/Biomarkers/images/tree-graphic.png", alt="tree-graphic">
</div>


<div class="alert alert-block alert-success">
<b>&#9997; Reference:</b> <a href="https://www.mastersindatascience.org/learning/machine-learning-algorithms/decision-tree/">https://www.mastersindatascience.org/learning/machine-learning-algorithms/decision-tree/</a>
</div>

As we mentioned before, decision trees can predict outcomes when asked a direct question such as "based on the following characteristics within my dataset is today more likely to be  rainy or sunny?". In our case our question would be "based on the characteristics within our dataset which biomarkers would lead to an injured or healthy sample?". As the tree goes down its branches it will ask more questions such as "does the sample also show that this biomarker is present? What is the concentration?" until it has narrowed down its decision to whether the sample is healthy or injured.

<div class="alert alert-block alert-danger">
<b>&#128721; Caution:</b> Although decision trees are great for the question we are asking they are not the best to use in large data sets or for complex questions that deal with may factors. They are also prone to overfitting and can give a low prediction accuracy rate.
</div>

## Creating a Decision Tree

Before we run our decision tree we need to set up the training procedure. In the command below we are setting a K-fold cross validation that we have set to 10 and we ask to repeat that 3 times. To understand this, think of a deck of cards as our dataset. We will be breaking our deck into 10 groups (these are our K folds).  We will train and test all sets of cards then repeat the process again another two times by shuffling our deck and again breaking the cards into 10 sets per shuffle. This will help to ensure that each iteration is different from the next which helps to avoid overfitting and helps to obtain a higher accuracy rate.

In [ ]:
# Set seed so stats don't change every time this analysis is run
set.seed(825)

# Set up the training procedure or train control object, repeated Cross-Validation, that you'll refer to later in the training model
# Below you'll see we chose to do 10-fold cross-validation repeated 3 times 
train.control_dt <- trainControl(
                           method = "repeatedcv",
                           number = 10, ## 10-fold CV
                           repeats = 3,## repeated three times
                           )

Now we need to select our tuning parameter. Each model has a hyperparameter that can be tuned to optimize your model's architecture and get the highest rate of performance possible (aka greater accuracy). The major tuning parameter for decision trees is complexity parameter (cp).  This is the minimum improvement amount, i.e. how many branches can the tree make without increase relative error. Below we've set it to a range because adding too large of a value will result in a very small tree and too small of a value can result in overfitting and too many branches in the tree.

In [ ]:
# Save parameters in tune grid object that you'll refer to later in the training model
tune.grid_dt <- expand.grid(cp = seq(from = 0, to = .5, by = 0.001))

Now we can finally train our model. We will be using the **rpart** function within the **caret** package to train our decision tree based on the **'D'** column within our training dataset. You can see that we have added our parameters that we created before: training procedures and setting our cp grid to gain the best accuracy rate.

In [ ]:
model_train_dt <- train(D~., data = data.frame(training), 
                 method = "rpart", 
                 tuneGrid = tune.grid_dt,
                 trControl = train.control_dt,
                 metric = "Accuracy"
               )

In [ ]:
plot(model_train_dt)

Now we tune our model with the best tuning parameter.

In [ ]:
# Show best tuning parameter from all the ones we tested
# Print the best tuning parameter cp that maximizes model accuracy
model_train_dt$bestTune

# Show the model 
model_train_dt


From our plot and the table we can see that the best cp value with the highest accuracy of 75.3% is 0.222.

We can plot our tree based on our training dataset by using the **rattle** package to add some color. Looks like it found a simple pattern, samples that have biomarkers Thbs at certain concentrations present may indicate injury. 

To explain the number within the decision nodes if the protein Thbs is present in the sample and the concentration is less than -0.52 mg/dL then there is a .73% chance that the sample is healthy. But if it has a greater concentration then there is a 27% chance the sample will be injured. The overall prediction for this biomarker has a 100% accuracy.

In [ ]:
fancyRpartPlot(model_train_dt$finalModel)

Now we can finally test our model using the **predict** function. You will notice that we run **predict** twice.  The second time is to gather all prediction possibilities that we can later plot in our ROC/AUC curves. We will also create a <b>Confusion Matrix</b> that evaluates our models by showing us how many predictions were true positives, false positives, true negatives, and false negatives. We will go into more detail about confusion matrices towards the end of this submodule.

In [ ]:
# Testing - Returns Predictions
model_test_dt <- predict(object = model_train_dt,
          newdata = data.frame(testing))
# Testing - Returns Probabilities used in ROC/AUC curves
model_test_probs_dt <- predict(object = model_train_dt,
          newdata = data.frame(testing), type = "prob")


# Print confusion matrix  
confusionMatrix(data = model_test_dt,
                  reference = testing$D,
                  positive = "Injured")

Our results show that our model predicted 4 Injured samples as true positive, and the overall accuracy of our model was 79%.

---

<div class="alert alert-block alert-warning">
<b>&#127891; Note:</b> The remaining models follow the same steps as the decision tree. Though they do use different methods and tuning parameters.
</div>


## Random Forest Model

<b>Random Forests</b> consists of hundreds or thousands of decision trees, each containing different outcomes or predictions. It then takes the average of these trees based on their accuracy to present the best prediction with the least amount of errors. Random forests have higher accuracy and can answer complex questions like "based on the following characteristics within my dataset will today more likely be a rainy, sunny, cloudy, or windy day?".

Random forests use **mtry** to tune their models. Since the trees are made independently from each other we need to create some randomness and thats exactly what mtry does!

In [ ]:
# Set seed so stats don't change every time this analysis is run
set.seed(123)

# SELECT TUNING PARAMETERS
# The major tuning parameter for the random forest model is mtry. This determines how many predictors are used at each split.
# A common value for default is the square root of the number of predictors in software packages. 

# Define mtry values you'd like to assess
sqrt(ncol(training))

mtry <- c(4.5, sqrt(ncol(training)), 6.5, 7.5, 8.5, 9.5)
          
# Save parameters in tune grid object that you'll refer to later in the training model
tunegrid <- expand.grid(.mtry=mtry)



<div class="alert alert-block alert-info">
<b>&#9995; Tip:</b>

If there are many relevant predictor variables, mtry should be set small so that the strongest influential variables are chosen in the splits alongside less influential variables, which can provide small but relevant performance gains. These less influential variables might, for example, be useful for the prediction of a small group of observations that stronger variables fail to predict correctly. When mtry is large less influential variables might not have the chance to contribute to prediction because stronger variables are preferably selected for splitting and thus “mask” the smaller effects. Genomics datasets often have a few relevant variables compared to many so it may be a good idea to try mtry values that are higher. Source: https://arxiv.org/pdf/1804.03515.pdf#:~:text=2.1.1%20Number%20of%20randomly,Lower%20values%20of%20mtry%20lead 
</div>

#### Random Forest Training

For random forest we are using the **rf** method from the **caret** package.

In [ ]:
# Set up the training procedure or train control object, repeated Cross-Validation, that you'll refer to later in the training model
# Below you'll see we chose to do 10-fold cross-validation repeated 3 times 
train_control_rf <- trainControl(
                           method = "repeatedcv",
                           number = 10,
                           repeats = 3)

# Fit the model 
model_train_rf <- train(D~., 
                      data=training, 
                      method='rf', 
                      metric='Accuracy', 
                      tuneGrid=tunegrid, 
                      trControl=train_control_rf)

# View the model
plot(model_train_rf)

# Show best tuning parameter from all the ones we tested
# Print the best tuning parameter C that maximizes model accuracy
model_train_rf$bestTune

# Show the model 
model_train_rf

We can see from our our model and the plot that when mtry = 8.5 we get the best accuracy (83.9%).

#### Optional: Taking a Closer Look at a Random Forest

As we know random forests are made up of hundreds of decision trees.  To help understand what one of them would look like, let's plot one! 

First we need to know which tree to choose. If we plot the final model the plot shows us the average error rate of our classes (Injured vs. Healthy). We can see that 500 decision trees have been created.

In [ ]:
# View output matrix to ensure the legend identifies the correct class
tail(plot(model_train_rf$finalModel))
# Plot OOB error plot
plot(model_train_rf$finalModel)
    legend("topright",fill=c("green","black", "red"), c( "Injured", "OOB error", "Healthy"))

We can map any of these trees from the plot above but it's best to map one that has a low error value. Tree 450 looks like it has low error rates for both classes.

Before we plot anything we first need to set the final model of our random forest then we use the **forest** function to visualize the tree. 

In [ ]:
final_model <- model_train_rf$finalModel$forest

Next we will run the command below to extract and structure our tree.

In [ ]:
tree_func <- function(final_model, 
                      tree_num) {
  
  # Get tree by index
  tree <- randomForest::getTree(final_model, 
                                k = tree_num, 
                                labelVar = TRUE) %>%
    tibble::rownames_to_column() %>%
    # Make leaf split points to NA, so the 0s won't get plotted
    mutate(`split point` = ifelse(is.na(prediction), `split point`, NA))
  
  # Prepare data frame for graph
  graph_frame <- data.frame(from = rep(tree$rowname, 2),
                            to = c(tree$`left daughter`, tree$`right daughter`))
  
  # Convert to graph and delete the last node that we don't want to plot
  graph <- graph_from_data_frame(graph_frame) %>%
    delete_vertices("0")
  
  # Set node labels
  V(graph)$node_label <- gsub("_", " ", as.character(tree$`split var`))
  V(graph)$leaf_label <- as.character(tree$prediction)
  V(graph)$split <- as.character(round(tree$`split point`, digits = 2))
  
  # Plot
  plot <- ggraph(graph, 'dendrogram') + 
    theme_bw() +
    geom_edge_link() +
    geom_node_point() +
    geom_node_text(aes(label = node_label), na.rm = TRUE, repel = TRUE) +
    geom_node_label(aes(label = split), vjust = 2.5, na.rm = TRUE, fill = "white") +
    geom_node_label(aes(label = leaf_label, fill = leaf_label), na.rm = TRUE, 
                    repel = TRUE, colour = "white", fontface = "bold", show.legend = FALSE) +
    theme(panel.grid.minor = element_blank(),
          panel.grid.major = element_blank(),
          panel.background = element_blank(),
          plot.background = element_rect(fill = "white"),
          panel.border = element_blank(),
          axis.line = element_blank(),
          axis.text.x = element_blank(),
          axis.text.y = element_blank(),
          axis.ticks = element_blank(),
          axis.title.x = element_blank(),
          axis.title.y = element_blank(),
          plot.title = element_text(size = 18))
  
  print(plot)
}

Here we are setting the tree number to the number we decided before and voila! 

In [ ]:
tree_num=450 # Tree number
suppressWarnings(tree_func(final_model =  model_train_rf$finalModel, tree_num)) #displaying the plot

#### Random Forest Testing

In [ ]:
# Testing - Returns Predictions
model_test_rf <- predict(object = model_train_rf,
          newdata = testing)
# Testing - Returns Probabilities used in ROC/AUC curves
model_test_probs_rf <- predict(object = model_train_rf,
          newdata = testing, type = "prob")


# Print confusion matrix  
confusionMatrix(data = model_test_rf,
                  reference = testing$D,
                  positive = "Injured")

Looking at the confusion matrix our random forest model was able to predict one true positive for injured samples and has a accuracy of 73.7%.

---

## GBM Model
<b>Gradient Boosting Machines (GBM)</b> are similar to random forests in that they both consist of hundreds of decision trees. The difference between the two is that GBM models create one tree at a time so that each tree can build on each other by looking at the previous one and learning where the errors lie, leading to the next tree having better accuracy with its prediction. As we mentioned before, random forests create trees individually by randomly selecting which features the tree should contain then asses all trees together to see how to get the higher accuracy rate. 

We are using **carets** default for tuning our GBM model.

#### GBM Training 

In [ ]:
# Set seed so stats don't change every time this analysis is run
set.seed(156)

# Set up the training procedure or train control object, repeated Cross-Validation, that you'll refer to later in the training model
# Below you'll see we chose to do 10-fold cross-validation repeated 3 times 

train_control_gbm <- trainControl(## 10-fold CV
                           method = "repeatedcv",
                           number = 10,
                           ## Repeated ten times
                           repeats = 10)

# Fit the model 
model_train_gbm <- train(D ~ ., data = training, 
                 method = "gbm", 
                 trControl = train_control_gbm,
                 ## This last option is actually one
                 ## for gbm() that passes through
                 verbose = FALSE)

# View the model
plot(model_train_gbm)

# Show best tuning parameter from all the ones we tested
# Print the best tuning parameter interaction.depth that maximizes model accuracy
model_train_gbm$bestTune

# Show the model 
model_train_gbm

We can see that tuning GBM has four factors instead of one compared to the other models. 

- **n.trees** - Number of iterations
- **interaction.depth** - How complex the tree should be
- **shrinkage** - How quickly the model can adapt to get a lower error rate each time a tree is created
- **n.minobsinnode** - The minimum number of observations in each node this can limit the number of times the nodes can split.

#### GBM Testing 

In [ ]:
# Testing
model_test_gbm <- predict(object = model_train_gbm,
          newdata = testing)

model_test_probs_gbm <- predict(object = model_train_gbm,
          newdata = testing, type = "prob")

# Print confusion matrix  
confusionMatrix(data = model_test_gbm,
                  reference = testing$D,
                  positive = "Injured")

Our GBM model was able to predict two true positive injured sample with a 84% accuracy.

---

## SVM Model

<b>Support Vector Machine (SVM)</b> models are similar to decision trees in the sense that they are regularly used for classification problems where the data is sparse and work well when looking at two classes. Classification is performed by finding the hyper-plane that best differentiates between the two classes. For SVM we are using the **svmLinear** method from the **caret** package.

The hyperparameter to tune for SVM models is the regularization parameter (C).  This parameter adds a penalty every time an extreme parameter creates overfitting, which is usually related to large C values.  This is why it's better to set a range of small values to tune your model. 

#### SVM Training 

In [ ]:
# Set seed so stats don't change every time this analysis is run
set.seed(2019)

# SELECT TUNING PARAMETERS
# The major tuning parameter for the SVM Linear model is Cost. It essentially imposes a penalty to the model for making an error. 
# The higher the value of C, the less likely it is that the SVM algorithm will misclassify a point.

# Make a list of cost penalty values you'de like to assess
costs_svm <- c(0.25, 0.4, 0.5, 0.6, 1, 1.2, 1.8,2.0)

# Save parameters in tune grid object that you'll refer to later in the training model
# To skip the step above and generate a list you could use: tune_grid_svm <- expand.grid(C = seq(0, 4, length = 20))
tune_grid_svm <- expand.grid(C = costs_svm)

# Set up train control object, k-fold Cross-Validation, that you'll refer to later in the training model
train_control_svm <- trainControl(method="repeatedcv", 
                                  number=10, 
                                  repeats=3,
                                  savePredictions = TRUE,
                                  classProbs = TRUE)

# Fit the model 
model_train_svm <- train(D~., 
              data = training, 
              method = "svmLinear", 
              tuneGrid = tune_grid_svm,
              trControl = train_control_svm,  
              preProcess = c("center","scale"),
              metric='Accuracy')

# View the model
plot(model_train_svm)

# Show best tuning parameter from all the ones we tested
# Print the best tuning parameter C that maximizes model accuracy
model_train_svm$bestTune

# Show the model 
model_train_svm

We can see from the graph and the table that the best value for C is 1.8 and it also has the best accuracy of 79.7%.

#### SVM Testing 

In [ ]:
# Testing
model_test_svm <- predict(object = model_train_svm,
          newdata = testing)

model_test_probs_svm <- predict(object = model_train_svm,
          newdata = testing, type = "prob")


# Print confusion matrix  
confusionMatrix(data = model_test_svm,
                  reference = testing$D,
                  positive = "Injured")

Testing our SVM model shows that it was able to predict one true positive injured sample and has a 73.7% accuracy.

---

# Comparing Our Models With Metrics

In the previous section, we printed confusion matrices.  Here we give a deeper explanation of confusion matrices and ROC/AUC plots. 

<div>
  <img src="https://raw.githubusercontent.com/riinbre-bioinfo/Colab_Biomarkers/main/Biomarkers/images/ConfusionMatrix.png", alt="Confusion Matrix">
</div>


Above we define a confusion matrix and common metrics calculated from its components. Most are familiar with the terms true positives vs. false positives and true negatives vs. false negatives. In our case a true positive is when we are labeling or predicting an injured sample as injured whereas a false positive is when we are labeling a sample that is really healthy as injured. ROC curves plot the false positive rate against the true positive rate. True positive rate is also known as sensitivity while false positive rate = (1-specificity). The line on a ROC curve represents the ROC while the area under the curve is the AUC and is an indicator of performance. A higher AUC indicates better performance of the model. Here we are plotting all four machine learning models to compare their performances.

In [ ]:
# Create ROC Curve for all models

pROC_DT <- roc(testing$D, predictor = model_test_probs_dt[, "Injured"], plot=TRUE, legacy.axes=TRUE, main="Decision Tree", xlab="False Positive Rate", ylab="True Postive Rate", col="#c45c12", lwd=4, print.auc=TRUE)
pROC_RF <- roc(testing$D, predictor = model_test_probs_rf[, "Injured"], plot=TRUE, legacy.axes=TRUE, main="Random Forest", xlab="False Positive Rate", ylab="True Postive Rate", col="#4daf4a", lwd=4, print.auc=TRUE)
pROC_GBM <- roc(testing$D, predictor = model_test_probs_gbm[, "Injured"], plot=TRUE, legacy.axes=TRUE,  main="GBM", xlab="False Positive Rate", ylab="True Postive Rate", col="#764aaf", lwd=4, print.auc=TRUE)
pROCSVM <- roc(testing$D, predictor = model_test_probs_svm[, "Injured"], plot=TRUE, legacy.axes=TRUE, main="SVM", xlab="False Positive Rate", ylab="True Postive Rate", col="#377eb8", lwd=4, print.auc=TRUE)

Above, we see that the model with the highest AUC was our Random Forest with a value of 0.914.

# Conclusion

Overall we can see that our Random Forest gave the best performance having been able to predict two true positive injured samples and having a AUC of 0.914. This goes to show that even though the decision tree was able to correctly predict four injured samples our graph shows it had the worst AUC out of our models. Our SVM and GBM models tied with a AUC of 0.9 though this is a high score looking at the confusion matrix after testing the models showed that they only predicted one true positive injured sample. This means that the models guessed mostly healthy because they noticed that this resulted in a lower error rate not because they truly understood how to predict injured samples, that is another reason why they were not good models to use for our dataset.

<p><span style="font-size: 30px"><b>Quizzes</b></span> <span style="float : inline;">(run the command below to display the quizzes)</span> </p>

In [ ]:
IRdisplay::display_html('<iframe src="quizes/Chapter9_Quizes.html" width=100% height=450></iframe>')

---

# The End!

Thank you for completing this module!  We hope you found it helpful and encourage you to look at other modules available in the NIH sandbox.  If you have any comments on this module, please send an email to hemmecl@uri.edu (please put NOSI Biomarker Project in the subject line).

This project was funded as an administrative supplement to the Rhode Island INBRE grant P20GM10343 and would not have been possible without the help of numerous individuals.  Specifically:

- Dr. Nisanne Ghonem and her laboratory the the Department of Biomedical and Pharmaceutical Sciences, College of Pharmacy, University of Rhode Island for allowing us to use their data for this module.
- Dr. Bongsup Cho, Program Director of RI-INBRE
- Lakshmi Matukumalli, Program Director, Networks and Development Programs, NIGMS/NIH for providing this opportunity
- The teams at Google and Deloitte (Laura Beaudry, J.T. Turner, Daniel Pan, Allen Kim, Zelaikha Yosufzai, Marcia Price) who provided much needed technical support in developing this module

---